In [247]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

In [248]:
with open('../data/k2pandc_2025.10.03_13.31.02.csv') as k2panda:
    k2panda_df = pd.read_csv(k2panda)
k2panda_df

,pl_name,hostname,default_flag,disposition,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,...,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249,2018-04-25,2018-03,2018-02-15
1,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,...,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249,2018-04-25,2016-10,2016-07-28
2,BD+20 594 b,BD+20 594,1,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,K2,...,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249,2018-04-25,2017-03,2018-04-26
3,EPIC 201111557.01,EPIC 201111557,1,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,K2,...,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307,2018-08-02,2018-08,2018-08-02
4,EPIC 201111557.01,EPIC 201111557,0,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,K2,...,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307,2018-02-15,2018-03,2018-02-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,WASP-85 A b,WASP-85 A,0,CONFIRMED,Mo&#x10D;nik et al. 2016,2,1,Transit,2016,K2,...,-0.010,8.733,0.026,-0.026,10.62410,0.000959,-0.000959,2019-09-05,2019-09,2019-09-05
4000,Wolf 503 b,Wolf 503,1,CONFIRMED,Peterson et al. 2018,1,1,Transit,2018,K2,...,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337,2022-05-23,2021-12,2022-05-23
4001,Wolf 503 b,Wolf 503,0,CONFIRMED,Peterson et al. 2018,1,1,Transit,2018,K2,...,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337,2018-09-04,2018-11,2018-09-06
4002,Wolf 503 b,Wolf 503,0,CONFIRMED,Peterson et al. 2018,1,1,Transit,2018,K2,...,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337,2023-04-17,2023-04,2023-04-17


In [249]:
np.where(pd.isnull(k2panda_df.iloc[0]))

(array([17, 18, 19, 20, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
        42, 43, 44, 45, 46, 47, 48, 49, 52]),)

In [250]:
k2panda_df.count()

pl_name           4004
hostname          4004
default_flag      4004
disposition       4004
disp_refname      4004
                  ... 
sy_gaiamagerr1    3948
sy_gaiamagerr2    3948
rowupdate         4004
pl_pubdate        4004
releasedate       4004
Length: 94, dtype: int64

In [251]:
k2panda_df.drop(['pl_name', 
                 'hostname', 
                 'default_flag', 
                 'disp_refname', 
                 'pl_refname', 
                 'releasedate', 
                 'pl_pubdate', 
                 'rowupdate', 
                 'soltype', 
                 'disc_year', 
                 'discoverymethod', 
                 'st_refname', 
                 'sy_refname',
                 'disc_facility',
                 'ttv_flag',
                 'pl_orbperlim',
                 'rastr'
                ], axis=1, inplace=True)

In [252]:
threshold = 0.8 * k2panda_df.shape[0]
k2panda_df = k2panda_df.dropna(thresh=threshold, axis=1)

In [253]:
for col in k2panda_df.columns:
    null_count = k2panda_df[col].isnull().sum()
    null_percentage = (null_count / len(k2panda_df)) * 100
    print(f"Column '{col}' has {null_count} null values ({null_percentage:.2f}%).")

Column 'disposition' has 0 null values (0.00%).
Column 'sy_snum' has 0 null values (0.00%).
Column 'sy_pnum' has 0 null values (0.00%).
Column 'pl_controv_flag' has 0 null values (0.00%).
Column 'pl_orbper' has 44 null values (1.10%).
Column 'st_rad' has 130 null values (3.25%).
Column 'st_raderr1' has 762 null values (19.03%).
Column 'st_raderr2' has 768 null values (19.18%).
Column 'st_radlim' has 130 null values (3.25%).
Column 'ra' has 0 null values (0.00%).
Column 'decstr' has 0 null values (0.00%).
Column 'dec' has 0 null values (0.00%).
Column 'sy_dist' has 125 null values (3.12%).
Column 'sy_disterr1' has 256 null values (6.39%).
Column 'sy_disterr2' has 256 null values (6.39%).
Column 'sy_vmag' has 42 null values (1.05%).
Column 'sy_vmagerr1' has 42 null values (1.05%).
Column 'sy_vmagerr2' has 42 null values (1.05%).
Column 'sy_kmag' has 23 null values (0.57%).
Column 'sy_kmagerr1' has 31 null values (0.77%).
Column 'sy_kmagerr2' has 31 null values (0.77%).
Column 'sy_gaiamag

In [254]:
"""
def parse_to_timedelta(time):
    if pd.isna(time):
        return None
    match = re.match(r'(\d{2})h(\d{2})m([\d.]+)s', time)

    if match:
        hours, minutes, seconds = match.groups()
        return timedelta(hours=int(hours), minutes=int(minutes), seconds=float(seconds))
    else:
        print(time)
    return None

k2panda_df['rastr'] = k2panda_df['rastr'].apply(parse_to_timedelta)
#k2panda_df['rastr'] = k2panda_df['rastr'].dt.total_seconds().astype(int)
k2panda_df['rastr']
"""

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
/tmp/nix-shell.idDH4T/ipykernel_2461/2171051615.py:1: SyntaxWarning: invalid escape sequence '\d'
  """


"\ndef parse_to_timedelta(time):\n    if pd.isna(time):\n        return None\n    match = re.match(r'(\\d{2})h(\\d{2})m([\\d.]+)s', time)\n\n    if match:\n        hours, minutes, seconds = match.groups()\n        return timedelta(hours=int(hours), minutes=int(minutes), seconds=float(seconds))\n    else:\n        print(time)\n    return None\n\nk2panda_df['rastr'] = k2panda_df['rastr'].apply(parse_to_timedelta)\n#k2panda_df['rastr'] = k2panda_df['rastr'].dt.total_seconds().astype(int)\nk2panda_df['rastr']\n"

In [255]:
def parse_dec_to_decimal(dec):
    if pd.isna(dec):
        return None
    match = re.match(r'([+-])(\d{2})d(\d{2})m([\d.]+)s', dec)

    if match:
        sign, degrees, minutes, seconds = match.groups()
        dd = int(degrees) + int(minutes)/60 + float(seconds)/3600
        if sign == '-':
            dd *= -1
        return dd
    return None

k2panda_df['decstr'] = k2panda_df['decstr'].apply(parse_dec_to_decimal)
k2panda_df['decstr']

/tmp/nix-shell.idDH4T/ipykernel_2461/1474457790.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  k2panda_df['decstr'] = k2panda_df['decstr'].apply(parse_dec_to_decimal)


0       20.599019
1       20.599019
2       20.599019
3       -6.268328
4       -6.268328
          ...    
3999     6.563783
4000    -6.139336
4001    -6.139336
4002    -6.139336
4003    -6.139336
Name: decstr, Length: 4004, dtype: float64

In [256]:
categorical_cols = k2panda_df.select_dtypes(include=['object']).columns.tolist()

#One Hot Encoding
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

cat_data = k2panda_df[categorical_cols].fillna('missing')
encoded = encoder.fit_transform(cat_data)

feature_names = encoder.get_feature_names_out(categorical_cols)
cat_df = pd.DataFrame(
    encoded,
    columns=feature_names,
    index=k2panda_df.index
)

#numeric columns
num_cols = k2panda_df.select_dtypes(include=['int64', 'float64', 'int32', 'float32', 'timedelta64']).columns

k2_encoded = pd.concat([
    cat_df,
    k2panda_df[num_cols]
], axis=1)

k2_encoded

,disposition_CANDIDATE,disposition_CONFIRMED,disposition_FALSE POSITIVE,disposition_REFUTED,sy_snum,sy_pnum,pl_controv_flag,pl_orbper,st_rad,st_raderr1,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2
0,0.0,1.0,0.0,0.0,1,1,0,41.688644,0.956122,0.098585,...,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249
1,0.0,1.0,0.0,0.0,1,1,0,41.685500,0.928000,0.055000,...,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249
2,0.0,1.0,0.0,0.0,1,1,0,41.685500,1.080000,0.060000,...,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249
3,1.0,0.0,0.0,0.0,1,0,0,2.301830,0.762602,0.053651,...,-0.4598,11.727,0.046,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307
4,1.0,0.0,0.0,0.0,1,0,0,2.302368,0.710995,0.018609,...,-0.4598,11.727,0.046,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,0.0,1.0,0.0,0.0,2,1,0,2.655676,0.884000,0.083000,...,-1.2080,10.720,0.010,-0.010,8.733,0.026,-0.026,10.62410,0.000959,-0.000959
4000,0.0,1.0,0.0,0.0,1,1,0,6.001270,0.689000,0.021000,...,-0.0961,10.270,0.030,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337
4001,0.0,1.0,0.0,0.0,1,1,0,6.001180,0.690000,0.025000,...,-0.0961,10.270,0.030,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337
4002,0.0,1.0,0.0,0.0,1,1,0,6.001270,0.689000,0.021000,...,-0.0961,10.270,0.030,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337


In [259]:
imputer = KNNImputer(n_neighbors=5)
k2_imputed = pd.DataFrame(
    imputer.fit_transform(k2_encoded),
    columns=k2_encoded.columns
)
k2_imputed

,disposition_CANDIDATE,disposition_CONFIRMED,disposition_FALSE POSITIVE,disposition_REFUTED,sy_snum,sy_pnum,pl_controv_flag,pl_orbper,st_rad,st_raderr1,...,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,41.688644,0.956122,0.098585,...,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249
1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,41.685500,0.928000,0.055000,...,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249
2,0.0,1.0,0.0,0.0,1.0,1.0,0.0,41.685500,1.080000,0.060000,...,-1.2400,10.849,0.012,-0.012,9.368,0.018,-0.018,10.86440,0.000249,-0.000249
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.301830,0.762602,0.053651,...,-0.4598,11.727,0.046,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.302368,0.710995,0.018609,...,-0.4598,11.727,0.046,-0.046,9.220,0.019,-0.019,11.39950,0.001307,-0.001307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,0.0,1.0,0.0,0.0,2.0,1.0,0.0,2.655676,0.884000,0.083000,...,-1.2080,10.720,0.010,-0.010,8.733,0.026,-0.026,10.62410,0.000959,-0.000959
4000,0.0,1.0,0.0,0.0,1.0,1.0,0.0,6.001270,0.689000,0.021000,...,-0.0961,10.270,0.030,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337
4001,0.0,1.0,0.0,0.0,1.0,1.0,0.0,6.001180,0.690000,0.025000,...,-0.0961,10.270,0.030,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337
4002,0.0,1.0,0.0,0.0,1.0,1.0,0.0,6.001270,0.689000,0.021000,...,-0.0961,10.270,0.030,-0.030,7.617,0.023,-0.023,9.89816,0.000337,-0.000337


In [272]:
y = k2_imputed[[
    'disposition_CONFIRMED',
    'disposition_CANDIDATE',
    'disposition_FALSE POSITIVE',
    'disposition_REFUTED'
]]
X = k2_imputed.drop(columns=['disposition_CONFIRMED', 
                             'disposition_CANDIDATE', 
                             'disposition_FALSE POSITIVE',
                             'disposition_REFUTED'
                            ])

In [273]:
#dropping columns 
corr_matrix = X.corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.6)]
X = X.drop(columns=to_drop)
print(f"Dropping columns from high correlation: {to_drop}")

Dropping columns from high correlation: ['st_raderr2', 'decstr', 'dec', 'sy_disterr1', 'sy_disterr2', 'sy_vmagerr2', 'sy_kmag', 'sy_kmagerr2', 'sy_gaiamag', 'sy_gaiamagerr2']


In [274]:
k2_full = pd.concat([y, X], axis=1)
k2_full

,disposition_CONFIRMED,disposition_CANDIDATE,disposition_FALSE POSITIVE,disposition_REFUTED,sy_snum,sy_pnum,pl_controv_flag,pl_orbper,st_rad,st_raderr1,st_radlim,ra,sy_dist,sy_vmag,sy_vmagerr1,sy_kmagerr1,sy_gaiamagerr1
0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,41.688644,0.956122,0.098585,0.0,53.651123,179.4610,10.849,0.012,0.018,0.000249
1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,41.685500,0.928000,0.055000,0.0,53.651123,179.4610,10.849,0.012,0.018,0.000249
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,41.685500,1.080000,0.060000,0.0,53.651123,179.4610,10.849,0.012,0.018,0.000249
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.301830,0.762602,0.053651,0.0,183.846245,97.1795,11.727,0.046,0.019,0.001307
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.302368,0.710995,0.018609,0.0,183.846245,97.1795,11.727,0.046,0.019,0.001307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3999,1.0,0.0,0.0,0.0,2.0,1.0,0.0,2.655676,0.884000,0.083000,0.0,175.908033,141.8870,10.720,0.010,0.026,0.000959
4000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,6.001270,0.689000,0.021000,0.0,206.846198,44.5260,10.270,0.030,0.023,0.000337
4001,1.0,0.0,0.0,0.0,1.0,1.0,0.0,6.001180,0.690000,0.025000,0.0,206.846198,44.5260,10.270,0.030,0.023,0.000337
4002,1.0,0.0,0.0,0.0,1.0,1.0,0.0,6.001270,0.689000,0.021000,0.0,206.846198,44.5260,10.270,0.030,0.023,0.000337


In [275]:
k2_full.to_csv('../data/k2panda_fin.csv')